In [1]:
"""
ID - ID da partida
Legenda - campeonato-brasileiro-full.csv
Rodada : Rodada que aconteceu a partida
Data : Data que ocorreu a partida
Horário :  Horário que ocorreu a partida
Dia : Dia da semana que ocorreu a partida
Mandante : Clube mandante                      
Visitante : Clube Visitante                      
Vencedor : Clube vencedor da partida. Quando tiver "-", é um empate
Arena : Arena que ocorreu a partida                      
Mandante Placar : Gols que o clube mandante fez na partida                      
Visitante Placar : Gols que o clube visitante fez na partida                      
Estado Mandante : Estado do clube mandatorio                      
Estado Visitante : Estado do clube visitante                      
Estado Vencedor : Estado do clube vencedor. Quando tiver "-", é um empate


Legenda - campeonato-brasileiro-estatisticas-full.csv
ID - ID da partida
Mandante - 0 == Não (Visitante), 1 == Sim (Mandante)
Chutes - Finalizações
Chutes a gol - Finalizações na direção do gol
Posse de bola - Percentual da posse de bola
Passes - Quantidade de passes que o clube deu na partida
Precisão de Passe - Percentual da precisão de passe
Faltas - Quantidade de faltas cometidas na partida
Cartões Amarelo - Quantidade de cartões amarelos para o clube na partida
Cartões Vermelhos - Quantidade de cartões vermelhos para o clube na partida
Impedimentos - Quantidade de impedimentos para o clube na partida
Escanteios - Quantidade de escanteios para o clube na partida

"""


'\nID - ID da partida\nLegenda - campeonato-brasileiro-full.csv\nRodada : Rodada que aconteceu a partida\nData : Data que ocorreu a partida\nHorário :  Horário que ocorreu a partida\nDia : Dia da semana que ocorreu a partida\nMandante : Clube mandante                      \nVisitante : Clube Visitante                      \nVencedor : Clube vencedor da partida. Quando tiver "-", é um empate\nArena : Arena que ocorreu a partida                      \nMandante Placar : Gols que o clube mandante fez na partida                      \nVisitante Placar : Gols que o clube visitante fez na partida                      \nEstado Mandante : Estado do clube mandatorio                      \nEstado Visitante : Estado do clube visitante                      \nEstado Vencedor : Estado do clube vencedor. Quando tiver "-", é um empate\n\n\nLegenda - campeonato-brasileiro-estatisticas-full.csv\nID - ID da partida\nMandante - 0 == Não (Visitante), 1 == Sim (Mandante)\nChutes - Finalizações\nChutes a gol 

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row



spark = SparkSession.builder\
    .master('local')\
    .appName("Analise Dados Brasileirão")\
    .config('spark.ui.port','4050')\
    .getOrCreate()


In [3]:
#carregando dataset de estatistica campeonato brasileiro

stsFull = spark.read.csv("campeonato-brasileiro-estatisticas-full.csv",
                         inferSchema=True, 
                         header=True,
                         sep=';')
#carregando dataset de resultado jogos campeonato brasileiro

sts = spark.read.csv("campeonato-brasileiro-full.csv",
                     inferSchema=True, 
                     header=True,
                     sep=';')
#criando view para trartamento dos dados via sql spark

stsFull.createOrReplaceTempView('vwSTSfull')
sts.createOrReplaceTempView('vwSTS')

In [4]:

#criando  dataset para tratamento da informação
estatisticaBrasileirao = spark.sql("""
        select distinct
             x.id, x.rodada, x.dia, x.mandante, x.visitante, x.vencedor,x.data,
             y.chutes, y.chutes_a_gol, y.passes, y.posse_de_bola,
             case
                 when y.mandante = 1 then y.chutes end chutes_mandante,
             case
                 when y.mandante = 1 then y.chutes_a_gol end chutes_gol_mandante,
             case
                 when y.mandante = 1 then y.passes end passes_mandante,
             case
                 when y.mandante = 1 then y.posse_de_bola end posse_bola_mandante,
             case
                 when y.mandante = 0 then y.chutes end chutes_visitante,
             case
                 when y.mandante = 0 then y.chutes_a_gol end chutes_gol_visitante,
             case
                 when y.mandante = 0 then y.passes end passes_visitante,
             case
                 when y.mandante = 0 then y.posse_de_bola end posse_bola_visitante
                 
        from vwSTS x
            join vwSTSfull y on
             x.id = y.id
""")

In [5]:
estatisticaBrasileirao.printSchema()

root
 |-- id: integer (nullable = true)
 |-- rodada: string (nullable = true)
 |-- dia: string (nullable = true)
 |-- mandante: string (nullable = true)
 |-- visitante: string (nullable = true)
 |-- vencedor: string (nullable = true)
 |-- data: string (nullable = true)
 |-- chutes: integer (nullable = true)
 |-- chutes_a_gol: integer (nullable = true)
 |-- passes: integer (nullable = true)
 |-- posse_de_bola: string (nullable = true)
 |-- chutes_mandante: integer (nullable = true)
 |-- chutes_gol_mandante: integer (nullable = true)
 |-- passes_mandante: integer (nullable = true)
 |-- posse_bola_mandante: string (nullable = true)
 |-- chutes_visitante: integer (nullable = true)
 |-- chutes_gol_visitante: integer (nullable = true)
 |-- passes_visitante: integer (nullable = true)
 |-- posse_bola_visitante: string (nullable = true)

